In [ ]:
import numpy as np 
import pandas as pd 
import yfinance as yf 
import matplotlib.pyplot as plt 

TICKER = "TSLA"

# Price Behavior 
INTERVAL = "1d"
if INTERVAL == "1h":
    PERIOD == "730d"
else:
    PERIOD == "max"

# MACD
MACD_FAST = 12
MACD_SLOW = 27
MACD_SPAN = 9

# MFI 
MFI_LENGTH = 14
OVERBROUGHT = 70
OVERSOLD = 30

# Data Subsection
LOOKBACK = 1000

def get_data(ticker = TICKER, interval = INTERVAL, lookback = LOOKBACK):
    
    # Step 1: Download Data
    df = yf.download(ticker, interval = interval, period = PERIOD)
    
    # Step 2: Select only Level 1 from Multilevel Column (MultiIndex)
    df.columns = df.columns.get_level_values(0)

    # Step 3: Simplifying df for Plots
    df = df.reset_index(drop=True)

    # Step 4: Return Subsect Data 
    return df.iloc[-lookback:, :]
        # -lookback: Get me the last lookback rows
        # : Get me all Columns

def add_MACD(df, fast = MACD_FAST, slow = MACD_SLOW, span = MACD_SPAN):
    
    # Step 1: Create Fast and Slow EMA Columns
    df[f]


